In [ ]:
# HÜCRE 1: Kütüphaneler ve Ayarlar
import torch
import torchaudio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from scipy.optimize import brentq
from scipy.interpolate import interp1d
from tqdm.notebook import tqdm
import time

# Senin model dosyan
from ecapa_model import ECAPA_TDNN 

# Orijinal Model için SpeechBrain
from speechbrain.inference.speaker import EncoderClassifier

# AYARLAR
PAIRS_FILE = "test_pairs_static.csv"   # Az önce ürettiğimiz dosya
MY_MODEL_PATH = "artifacts\model_0057.model"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Test Cihazı: {DEVICE}")

In [1]:
import os
import requests

url = "https://www.robots.ox.ac.uk/~vgg/data/voxceleb/meta/veri_test.txt"
save_path = "veri_test.txt"

print(f"İndiriliyor: {url}")
response = requests.get(url)

with open(save_path, "wb") as f:
    f.write(response.content)

print("✅ İndirme tamamlandı!")

İndiriliyor: https://www.robots.ox.ac.uk/~vgg/data/voxceleb/meta/veri_test.txt
✅ İndirme tamamlandı!


In [3]:
import os
import torch
import torchaudio
import numpy as np
from sklearn.metrics import roc_curve
from scipy.interpolate import interp1d
from scipy.optimize import brentq
from tqdm import tqdm
from ecapa_model import ECAPA_TDNN

# --- AYARLAR ---
# VoxCeleb1'in "wav" klasörünün olduğu ana yol
VOXCELEB_ROOT = r"C:\Users\omera\Desktop\speaker_verification\vox1_test_wav\wav"  
# Resmi test listesi (veri_test.txt) yolu
TEST_LIST_PATH = r"C:\Users\omera\Desktop\speaker_verification\veri_test.txt" 
# Senin Modelin
MODEL_PATH = r"C:\Users\omera\Desktop\speaker_verification\artifacts\model_0057.model"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_custom_model():
    print(f"Mini-ECAPA (C=256) Yükleniyor...")
    model = ECAPA_TDNN(C=256).to(device)
    checkpoint = torch.load(MODEL_PATH, map_location=device)
    
    # Prefix temizliği
    new_state = {}
    for k, v in checkpoint.items():
        if k.startswith("speaker_encoder."):
            new_state[k.replace("speaker_encoder.", "")] = v
    
    model.load_state_dict(new_state)
    model.eval()
    return model

def get_embedding(model, file_path):
    # Dosya yolu düzeltme (Windows/Linux slash farkı için)
    full_path = os.path.join(VOXCELEB_ROOT, file_path)
    
    try:
        wav, sr = torchaudio.load(full_path)
        
        # --- KRİTİK NOKTA: RESAMPLE ---
        # VoxCeleb 16kHz'dir. Senin modelin 24kHz.
        if sr != 24000:
            resampler = torchaudio.transforms.Resample(sr, 24000)
            wav = resampler(wav)
            
        # Mono yap
        if wav.shape[0] > 1: 
            wav = torch.mean(wav, dim=0, keepdim=True)
            
        wav = wav.to(device)
        
        with torch.no_grad():
            emb = model(wav, aug=False)
        return emb
        
    except Exception as e:
        print(f"Hata ({file_path}): {e}")
        return None

def run_voxceleb_benchmark():
    model = load_custom_model()
    
    # Listeyi oku
    print(f"Test listesi okunuyor: {TEST_LIST_PATH}")
    with open(TEST_LIST_PATH, "r") as f:
        lines = f.readlines()
        
    scores = []
    labels = []
    
    print(f"Toplam Test Çifti: {len(lines)}")
    print("Benchmark Başlıyor... (Bu işlem biraz sürebilir)")
    
    # TQDM ile ilerleme çubuğu
    for line in tqdm(lines):
        parts = line.strip().split()
        
        # Format genelde: [Label] [File1] [File2]
        label = int(parts[0])
        file1 = parts[1]
        file2 = parts[2]
        
        emb1 = get_embedding(model, file1)
        emb2 = get_embedding(model, file2)
        
        if emb1 is not None and emb2 is not None:
            score = torch.nn.CosineSimilarity(dim=-1)(emb1, emb2).item()
            scores.append(score)
            labels.append(label)
            
    # EER Hesaplama
    fpr, tpr, thresholds = roc_curve(labels, scores, pos_label=1)
    eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    thresh = interp1d(fpr, thresholds)(eer)
    
    print("\n" + "="*50)
    print("VOXCELEB1 TEST SONUÇLARI")
    print("="*50)
    print(f"Model: Custom Mini-ECAPA (24kHz, Clean-Trained)")
    print(f"✅ EER: %{eer*100:.2f}")
    print(f"Optimal Threshold: {thresh:.4f}")
    print("-" * 50)
    
    # Yorumlama Kılavuzu
    print("YORUM:")
    if eer*100 < 5.0:
        print("MÜKEMMEL! Temiz veriyle eğitilmene rağmen gürültülü veride harika çalışıyor.")
    elif eer*100 < 10.0:
        print("BAŞARILI. Domain Shift (Dil ve Kayıt farkı) makul seviyede.")
    else:
        print("NORMAL. Modelin sadece temiz Türkçe sese özelleşmiş (Domain Gap).")

if __name__ == "__main__":
    if os.path.exists(VOXCELEB_ROOT) and os.path.exists(TEST_LIST_PATH):
        run_voxceleb_benchmark()
    else:
        print("HATA: Klasör yollarını kontrol et!")
        print(f"Wav Klasörü: {VOXCELEB_ROOT}")
        print(f"Test Listesi: {TEST_LIST_PATH}")

Mini-ECAPA (C=256) Yükleniyor...
Test listesi okunuyor: C:\Users\omera\Desktop\speaker_verification\veri_test.txt
Toplam Test Çifti: 37720
Benchmark Başlıyor... (Bu işlem biraz sürebilir)


  0%|          | 0/37720 [00:00<?, ?it/s]C:\Users\omera\AppData\Roaming\Python\Python311\site-packages\torchaudio\_backend\utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
100%|██████████| 37720/37720 [33:13<00:00, 18.92it/s]



VOXCELEB1 TEST SONUÇLARI
Model: Custom Mini-ECAPA (24kHz, Clean-Trained)
✅ EER: %21.95
Optimal Threshold: 0.3523
--------------------------------------------------
YORUM:
NORMAL. Modelin sadece temiz Türkçe sese özelleşmiş (Domain Gap).


In [ ]:
from ecapa_tdnn_original import ECAPA_TDNN

orig_model = load_custom_model()

Mini-ECAPA (C=256) Yükleniyor...


In [9]:
import torch
import torchaudio
import pandas as pd
import numpy as np
import os
from sklearn.metrics import roc_curve
from scipy.optimize import brentq
from scipy.interpolate import interp1d
from tqdm import tqdm

# --- AYARLAR ---
# Daha önce oluşturduğun sabit test çiftleri dosyası
PAIRS_FILE = "test_pairs_static.csv" 
# Orijinal Model Kaynağı (Otomatik iner)
SOURCE = "speechbrain/spkrec-ecapa-voxceleb"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_custom_model():
    print(f"Mini-ECAPA (C=256) Yükleniyor...")
    model = ECAPA_TDNN(C=1024).to(device)
    checkpoint = torch.load(r"artifacts\pretrain.model", map_location=device)
    
    # Prefix temizliği
    new_state = {}
    for k, v in checkpoint.items():
        if k.startswith("speaker_encoder."):
            new_state[k.replace("speaker_encoder.", "")] = v
    
    model.load_state_dict(new_state)
    model.eval()
    return model

def get_embedding_original(model, wav_path):
    # Dosya kontrolü
    if not os.path.exists(wav_path):
        return None

    try:
        # Sesi yükle
        wav, sr = torchaudio.load(wav_path)
        
        # --- ÖNEMLİ: Orijinal model 16kHz ister ---
        if sr != 16000:
            resampler = torchaudio.transforms.Resample(sr, 16000)
            wav = resampler(wav)
            
        # Mono yap
        if wav.shape[0] > 1: 
            wav = torch.mean(wav, dim=0, keepdim=True)
            
        wav = wav.to(device)
        
        # Inference (SpeechBrain encode_batch bekler)
        with torch.no_grad():
            emb = model(wav,aug=False)
            
        # [1, 1, 192] -> [192] düzleştir
        return emb
        
    except Exception as e:
        print(f"Hata: {e}")
        return None

def run_benchmark():
    # 1. Modeli Yükle
    model = load_custom_model()
    
    # 2. Test Çiftlerini Oku
    if not os.path.exists(PAIRS_FILE):
        print(f"HATA: {PAIRS_FILE} bulunamadı! Önce generate_pairs kodunu çalıştır.")
        return

    df = pd.read_csv(PAIRS_FILE)
    print(f"Toplam Test Çifti: {len(df)}")
    
    scores = []
    labels = []
    
    print("🚀 Orijinal Model Testi Başlıyor...")
    
    for index, row in tqdm(df.iterrows(), total=len(df)):
        f1, f2, label = row['file1'], row['file2'], row['label']
        
        e1 = get_embedding_original(model, f1)
        e2 = get_embedding_original(model, f2)
        
        if e1 is not None and e2 is not None:
            # Cosine Similarity
            score = torch.nn.CosineSimilarity(dim=-1)(e1, e2).item()
            scores.append(score)
            labels.append(label)
            
    # EER Hesaplama
    fpr, tpr, thresholds = roc_curve(labels, scores, pos_label=1)
    eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    thresh = interp1d(fpr, thresholds)(eer)
    
    print("\n" + "="*50)
    print("ORİJİNAL MODEL (VOXCELEB) SONUÇLARI")
    print("="*50)
    print(f"Veri Seti: Senin Özel Türkçe Datasetin")
    print(f"✅ EER: %{eer*100:.2f}")
    print(f"Optimal Threshold: {thresh:.4f}")
    print("-" * 50)
    
    # Karşılaştırma Yorumu
    # Senin modelin %0.56 çıkmıştı.
    my_eer = 0.56 
    print("KARŞILAŞTIRMA:")
    print(f"Senin Modelin (Mini): %{my_eer:.2f}")
    print(f"Orijinal Model (Dev) : %{eer*100:.2f}")
    
    if my_eer < (eer*100):
        print("\n🏆 KAZANAN: SENİN MODELİN!")
        print("Yorum: Kendi sahanda (Domain Adaptation) devasa modeli yendin.")
    else:
        print("\n🏆 KAZANAN: ORİJİNAL MODEL")
        print("Yorum: Orijinal modelin genelleştirme yeteneği çok yüksekmiş.")

if __name__ == "__main__":
    run_benchmark()

Mini-ECAPA (C=256) Yükleniyor...
Toplam Test Çifti: 10000
🚀 Orijinal Model Testi Başlıyor...


  0%|          | 0/10000 [00:00<?, ?it/s]C:\Users\omera\AppData\Roaming\Python\Python311\site-packages\torchaudio\_backend\utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
100%|██████████| 10000/10000 [10:53<00:00, 15.30it/s]


ORİJİNAL MODEL (VOXCELEB) SONUÇLARI
Veri Seti: Senin Özel Türkçe Datasetin
✅ EER: %3.92
Optimal Threshold: 0.3170
--------------------------------------------------
KARŞILAŞTIRMA:
Senin Modelin (Mini): %0.56
Orijinal Model (Dev) : %3.92

🏆 KAZANAN: SENİN MODELİN!
Yorum: Kendi sahanda (Domain Adaptation) devasa modeli yendin.
